# Tabular-playground-series-nov-2022

## Details of competition
#### URL: https://www.kaggle.com/competitions/tabular-playground-series-nov-2022/overview

<br>

#### Topic
Provide a fun and approachable-for-anyone tabular dataset to model.

<br>

#### Background

<br>

##### About the Tabular Playground Series
Kaggle competitions are incredibly fun and rewarding, but they can also be intimidating for people who are relatively new in their data science journey. In the past, we've launched many Playground competitions that are more approachable than our Featured competitions and thus, more beginner-friendly.

<br>

**The goal of these competitions is to provide a fun and approachable-for-anyone tabular dataset to model.** These competitions are a great choice for people looking for something in between the Titanic Getting Started competition and the Featured competitions. If you're an established competitions master or grandmaster, these probably won't be much of a challenge for you; thus, we encourage you to avoid saturating the leaderboard.

<br>

For each monthly competition, we'll be offering Kaggle Merchandise for the top three teams. And finally, because we want these competitions to be more about learning, we're limiting team sizes to 3 individuals.

## 한 줄 요약!!

<br>

- **준비된 뒷토막 + 프로젝트 진행(앙상블 + 블랜딩)하는 앞토막 >> 모델 향상 >> 결과 제출**

## Files 항목

### submission_files/

- 앞 반토막에 대한 이진 분류 모델 예측값 파일 모음

<br>

### train_labels.csv

- 제출 파일에 대한 앞 반토막 실측값


<br>


### sample_submission.csv

- 뒷토막만 정상 데이터인 샘플 제출 파일


- 포맷(ids)은 정상임


- 점수 향상을 위해, 앞 반토막에 대해 추가 작업 진행 (데이터 섞기)




## 데이터 관련 이해사항:

<br>

### submission_files(folder)
- 이진 분류 작업에 대한 예측값들이 저장


- 각 파일 이름은 로그 손실(logloss) 점수


- 실측 라벨 파일(train_label?)의 예측 행의 절반에 대한 로그 손실 점수가 파일 제목


- 훌륭한 트레이닝 세트가 됨

<br>

### train_labels.csv

- submission_files의 파일의 행에 대한 실측 정보값 제공


<br>


### sample_submission.csv

- 이해가 어느정도 된 부분!

- 나머지 뒷부분 반토막(20000~39999)에 대한 결과 예측을 제출!!

## 우리의 목표

- 앙상블 활용!


- 다양한 submission file들의 앙상블을 통해, 모델 예측 성능을 높이는 것!!


- submission_files(반토막짜리 행에 대한 실측값들)들이 있으므로 블랜딩(여러 데이터를 합쳐서)하면 더 좋은 결과를 얻었는지 아닌지 알 수 있을 것임!!

<br>

### 요약

1. 두 파일을 합쳐서 반토막 데이터에 대한 점수(예측값)를 확인할 수 있음!!


2. 점수를 계속해서 개선해나갈 수 있음!!


3. 점수를 (극한으로) 개선 후, 나머지 행들에 대해, 리더보드에 제출할 수 있음!! 🥸



In [ ]:
# import
# 내일 진행 예정!!

In [1]:
# import module
import pandas as pd

In [4]:
# import data
# !ls
!ls ./tabular-playground-series-nov-2022/

sample_submission.csv submission_files      train_labels.csv


In [5]:
# import data
sample_submission = pd.read_csv('./tabular-playground-series-nov-2022/sample_submission.csv')
sample_submission

,id,pred
0,20000,0.640707
1,20001,0.636904
2,20002,0.392496
3,20003,0.588658
4,20004,0.783603
...,...,...
19995,39995,0.382515
19996,39996,0.352498
19997,39997,0.577554
19998,39998,0.712353


In [6]:
# import data
train_labels = pd.read_csv('./tabular-playground-series-nov-2022/train_labels.csv')
train_labels

,id,label
0,0,0
1,1,1
2,2,1
3,3,1
4,4,0
...,...,...
19995,19995,1
19996,19996,1
19997,19997,0
19998,19998,0


In [9]:
# import data
_6222863195 = pd.read_csv('./tabular-playground-series-nov-2022/submission_files/0.6222863195.csv')
_6222863195

,id,pred
0,0,0.709336
1,1,0.452988
2,2,0.675462
3,3,0.481046
4,4,0.957339
...,...,...
39995,39995,0.382515
39996,39996,0.352498
39997,39997,0.577554
39998,39998,0.712353
